In [1]:
import sys
sys.path.append('../src')

from extractor import MatrixFormattedGraph

In [2]:
%%time
mg = MatrixFormattedGraph('nodes.csv', 'edges.csv', start_kind='Compound', end_kind='Disease', max_length=4)

Reading file information...
Initializing metagraph...
Generating adjcency matrices...


100%|██████████| 24/24 [01:07<00:00,  6.19s/it]



Weighting matrices by degree with dampening factor 0.4...


100%|██████████| 25/25 [00:25<00:00,  1.03s/it]

CPU times: user 1min 46s, sys: 2.54 s, total: 1min 48s
Wall time: 1min 49s


In [3]:
%%time
res = mg.calculate_dwpc(start_nodes='Compound', end_nodes='Disease', n_jobs=16)

Calculating DWPCs...


100%|██████████| 1206/1206 [03:31<00:00,  2.22s/it]



Reformating results...


100%|██████████| 1206/1206 [04:38<00:00,  5.99it/s]


CPU times: user 6min 52s, sys: 2min 5s, total: 8min 58s
Wall time: 9min 44s


In [4]:
res.head()

,compound_id,disease_id,CdGuAeGuD,CtDdGdCpD,CbGr>GdDrD,CbG<rGuAlD,CbGuDrD,CtDuGcGdD,CuGiGdCpD,CcSEcCtDrD,...,CuGuCuGdD,CtDpCrCtD,CbG<rG<rGdD,CbGcGuD,CuGaDtCpD,CbGiGuAlD,CpDrDpCpD,CbGdDtCtD,CtDuGuCpD,CdG<rG<rGuD
0,DB00014,DOID:0050156,0.0,0.0,0.0,0.0,0.0,0.002994,0.0,0.024858,...,0.002064,0.0,0.0,0.0,0.0,0.000212,0.0,0.0,0.002600,0.0
1,DB00014,DOID:0050425,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.012142,...,0.000000,0.0,0.0,0.0,0.0,0.000767,0.0,0.0,0.003606,0.0
2,DB00014,DOID:0050741,0.0,0.0,0.0,0.0,0.0,0.001331,0.0,0.007261,...,0.001365,0.0,0.0,0.0,0.0,0.001953,0.0,0.0,0.000000,0.0
3,DB00014,DOID:0050742,0.0,0.0,0.0,0.0,0.0,0.005835,0.0,0.010915,...,0.003644,0.0,0.0,0.0,0.0,0.001106,0.0,0.0,0.000000,0.0
4,DB00014,DOID:0060073,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0


In [5]:
import bz2

In [6]:
with bz2.open('dwpc-features.tsv.bz2', 'wt') as write_file:
    res.to_csv(write_file, sep='\t', index=False, float_format='%.4g')